In [29]:
#create advanced DV reports
import astropy.io.fits as apf
import batman
import eleanor
import emcee
import getpass
import glob
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.backends.backend_pdf import PdfPages
from multiprocessing import Pool
import numpy as np
import os
import pandas as pd
import shutil
import time as tm
import math
import warnings
warnings.filterwarnings("ignore")
import numpy as np
# from tqdm.auto import tqdm
from tqdm.auto import tqdm

import ntpath


In [30]:
LDC_for_quadratic = pd.read_csv('./LDC_files/table15.dat', 
                                header = None, 
                                sep="\s+", index_col=None,
                               names = ['logg', 'Teff', 'z','L/HP', 'aLSM', 'bLSM',
                                       'aFCM', 'bFCM', 'SQRT(CHI2)', 'qsr', 'PC'])

mdwarf_LDC_for_quadratic = LDC_for_quadratic[LDC_for_quadratic['Teff']<4300]
                  
                  
                  

In [31]:
# mdwarf_LDC_for_quadratic[mdwarf_LDC_for_quadratic['Teff']==3300]

In [32]:
# # attempt_logg = 4.232
# attempt_Teff = 3550.0

# mdwarf_Teff = mdwarf_LDC_for_quadratic.iloc[(mdwarf_LDC_for_quadratic['Teff']-attempt_Teff-0.5).abs().argsort()[:8]].sort_values(['Teff']).reset_index(drop=True)
# mdwarf_Teff
# # mdwarf_logg = mdwarf_Teff.iloc[(mdwarf_Teff['logg']-attempt_logg).abs().argsort()[0]]

                                                                                                       

In [33]:
# total_mdwarfs_from_tic = pd.read_csv('../../Research_course/Tic_files/total_mdwarfs_from_tic.csv', 
#                                      nrows = 100,
#                                      header = 0, index_col = 0)


# total_mdwarfs_from_tic
                       


In [34]:
# total_mdwarfs_teff = pd.read_csv('../../Research_course/Tic_files/LDTK_params/mdwarfs_from_tic_after_run_90.csv', 
# #                                      nrows = 100,
#                                      header = 0, index_col = None)

# total_mdwarfs_teff['TICID'] = total_mdwarfs_teff['TICID'].astype(int)

# total_mdwarfs_teff = total_mdwarfs_teff.drop_duplicates(['TICID']).reset_index(drop=True)

In [35]:
# new_mdwarfs_GAIA = pd.read_csv('../../Research_course/Tic_files/GAIA/gaia_mdwarfs_from_tic_after_run_90.csv', 
#                                                                  header = 0, index_col = None)
# new_mdwarfs_GAIA['TICID'] = new_mdwarfs_GAIA['TICID'].astype(int)

# new_mdwarfs_GAIA = new_mdwarfs_GAIA.drop_duplicates(['TICID']).reset_index(drop=True)

In [36]:
# total_mdwarfs_df = total_mdwarfs_teff.merge(new_mdwarfs_GAIA, how = 'inner', on = ['TICID']).reset_index(drop=True)

# total_mdwarfs_df = total_mdwarfs_df[~(total_mdwarfs_df['Teff'].astype(float)>4200.).astype(bool)].reset_index(drop=True)

# total_mdwarfs_df

# # ticids = list(total_mdwarfs_df['TICID'])



In [37]:
LDC_for_quadratic = pd.read_csv('./LDC_files/table15.dat', 
                                header = None, 
                                sep="\s+", index_col=None,
                               names = ['logg', 'Teff', 'z','L/HP', 'aLSM', 'bLSM',
                                       'aFCM', 'bFCM', 'SQRT(CHI2)', 'qsr', 'PC'])

mdwarf_LDC_for_quadratic = LDC_for_quadratic[LDC_for_quadratic['Teff']<4300]
                  

def match_logg_and_teff_for_LDC(df):
    a = []
    b = []
#     print('a')
    bar_format = "{desc}{percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} targets | {elapsed}<{remaining}"
    pbar = tqdm(total=len(df), smoothing=0.3,  position=1, leave=True, bar_format=bar_format)
    for i in range(len(df)):
#         print(i)

        Teff = np.float128(df['Teff'])[i]
        logg =  np.float128(df['logg'])[i]
        pbar.update(1)

        try:
            int(Teff)
            mdwarf_Teff =mdwarf_LDC_for_quadratic[mdwarf_LDC_for_quadratic['Teff'] == np.median(mdwarf_LDC_for_quadratic.iloc[(mdwarf_LDC_for_quadratic['Teff'].astype('float128')-Teff+0.01).abs().argsort()[:8]].reset_index(drop=True)['Teff'])]
            if i%100 == 0:
    #                 print(Teff, set(mdwarf_Teff['Teff']))
                if len(set(mdwarf_Teff['Teff']))>1:
                       print(mdwarf_Teff)
            if not abs(logg)>=0.:
                logg = np.median(mdwarf_Teff['logg'])
    #         print(mdwarf_Teff.iloc[(mdwarf_Teff['logg'].astype('float128')-logg-0.00000001).abs().argsort()].iloc[0][['aLSM', 'bLSM']])
            aLSM, bLSM =  mdwarf_Teff.iloc[(mdwarf_Teff['logg'].astype('float128')-logg-0.00000001).abs().argsort()].iloc[0][['aLSM', 'bLSM']]
#             print(aLSM, bLSM)
            a.append(aLSM)
            b.append(bLSM)  

                    
        except:
            a.append(np.nan)
            b.append(np.nan)
    pbar.close()

    df['aLSM'] = a
    df['bLSM'] = b
    print(df[['aLSM', 'bLSM']])
    return df
    

In [38]:
# for i in range(1,13):
# # i=13
#     print(i)
#     filename = './Total_Mdwarf_files/by_sector/PM_year_1/mdwarf_params_sector_'+str(i)
#     df = pd.read_csv(filename, 
#                 header = 0, index_col = None)
    
# #     print(len(df))
# #     filename2 = './Total_Mdwarf_files/by_sector/PM_year_1/mdwarf_params_sector_'+str(i)+'.csv'
# #     df2 = pd.read_csv(filename,
# #                 header = 0, index_col = None)
# #     print(len(df2)==len(df))

#     df = df.drop(columns=['eTeff_y']).rename(columns = {'eTeff_x':'eTeff'})
#     print(df.columns)
#     new_df = match_logg_and_teff_for_LDC(df)
#     print(new_df[['aLSM', 'bLSM']])
#     print(new_df)
# #     new_df.to_csv(filename+'.csv')
    

In [39]:
# max_val=0
# lastname = None
# for name in glob.glob('./Total_Mdwarf_files/*'):
# #     print(ntpath.basename(name).split('_')[2])
#     n=int(os.path.splitext(ntpath.basename(ntpath.basename(name).split('_')[-1]))[0])
#     if n>max_val:
#         max_val=n
#         lastname = name
# print(max_val)

In [40]:
# lastname

In [41]:
# 322307100 in list(total_mdwarfs_df['TICID'])

In [42]:

# most_recent_file = lastname
# current_df = pd.read_csv(most_recent_file)

# # if max_val == 0:
# #     lst_of_dfs = []
# # else:
# #     lst_of_dfs = [current_df]

# lst_of_dfs = []


# total_mdwarfs_from_tic = pd.read_csv('../../Research_course/Tic_files/total_mdwarfs_from_tic.csv', 
#                                         chunksize = 10000, dtype = 'str', header = 0, index_col = 0)


# bar_format_2 = "{desc}{percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} targets | {elapsed}<{remaining}"
# pbar_2 = tqdm(total=397, smoothing=0.3,  position=0, leave=True, bar_format=bar_format_2)

# jjj = 0
# for chunk in total_mdwarfs_from_tic:
#     if jjj<max_val:
#         jjj+=1
#         pbar_2.update(1)

#         continue
#     else:
#     #     df = chunk[~(chunk['Teff'].astype(float)>4200.).astype(bool)]
#         df1 = chunk[chunk['TICID'].astype(int).isin(list(total_mdwarfs_df['TICID']))].reset_index(drop=True)
#     #     print(type(df1))
#         df2 = total_mdwarfs_df[total_mdwarfs_df['TICID'].astype(int).isin(df1['TICID'].astype(int))].reset_index(drop=True)
#         df2 = df2.astype(str)
#         df1 = df1.astype(str)

#     #     print(type(df2))
#         merged_df = df1.merge(df2, how = 'inner', on = ['TICID', 'Teff']).reset_index(drop=True).astype(float)
#         new_df = match_logg_and_teff_for_LDC(merged_df)

#         lst_of_dfs.append(new_df)
#         jjj+=1
#         final_df = pd.concat(lst_of_dfs).astype(str).reset_index(drop=True)
# #         print(max_val, jjj)
#         final_df.to_csv('./Total_Mdwarf_files/total_mdwarfs_'+str(max_val)+'_'+str(jjj))
#         pbar_2.update(1)

#     really_final_df = pd.concat(lst_of_dfs).astype(str).reset_index(drop=True)
# really_final_df.to_csv('./Total_Mdwarf_files/FINAL_mdwarf_params')
# print('FINISHED!!!!')
#     # tic_data = pd.concat(
#     #     [chunk[~(chunk['Teff'].astype(float)>4200.).astype(bool)]
#     #         for chunk in total_mdwarfs_from_tic]
#     #     ).reset_index(drop=True).astype(float)






In [43]:
# set_list = []
# for name in glob.glob('./Total_Mdwarf_files/*'):
# #     print(ntpath.basename(name).split('_')[2])
#     if len(ntpath.basename(name).split('_'))>3:
#         n=int(os.path.splitext(ntpath.basename(ntpath.basename(name).split('_')[2]))[0])
#         set_list.append(n)
# final_set = sorted(list(set(set_list)))

In [44]:
# finald_set

In [45]:
# final_set[28:]

In [46]:
# final_set.pop(2)
# final_set.pop(3
# final_set

In [47]:
# sorted(final_set)

# files = []
# lst1 = []
# max_val = 0
# # lastname = None

# # for name in glob.glob('./Total_Mdwarf_files/*'):
# # #     print(ntpath.basename(name).split('_')[2])
# #     if len(ntpath.basename(name).split('_'))<4:

# #         n=int(os.path.splitext(ntpath.basename(ntpath.basename(name).split('_')[2]))[0])
# #         if n>max_val:
# #             max_val=n
# #             lastname = name
# # print(max_val)
# # files.append(lastname)

# final_set = final_set[max_val:]
# #     else:
# for i in final_set:
#     for name in glob.glob('./Total_Mdwarf_files/*_'+str(i)+'_*'):

#         n=int(os.path.splitext(ntpath.basename(ntpath.basename(name).split('_')[-1]))[0])
#         if n>max_val:
#             max_val = n
#             lastname = name
#     files.append(lastname)

#     set_list.append(n)


In [48]:
# pd.read_csv(files[2])

In [49]:
# files

In [50]:
# pd.read_csv(files[1], index_col=0, skiprows=0)


In [51]:
# output_file = "./Total_Mdwarf_files/final_mdwarf_params.csv"
# # CHUNK_SIZE = 5000
# CHUNK_SIZE = 130000

# first_one = True
# for csv_file_name in files:

#     if first_one: # if it is not the first csv file then skip the header row (row 0) of that file
#         chunk_container = pd.read_csv(csv_file_name, chunksize=CHUNK_SIZE, index_col =0, header=0)
#     else:
# #         skip_row = 0

#         chunk_container = pd.read_csv(csv_file_name, chunksize=CHUNK_SIZE, index_col =0, header = 0, skiprows = 1)
#     for chunk in chunk_container:
#         chunk.to_csv(output_file, mode="a", index=False)
#     first_one = False


In [52]:
total_mdwarfs_ticids = pd.read_csv('./Total_Mdwarf_files/final_mdwarf_params.csv', usecols = ['TICID','S10'],
                                     header = 0).astype('str')
total_mdwarfs_ticids['TICID'] = total_mdwarfs_ticids['TICID'].astype(float)
total_mdwarfs_ticids = total_mdwarfs_ticids.sort_values(['TICID']).reset_index(drop=True)
set(total_mdwarfs_ticids['S10'])

{'0.0', '0.0.5', '0.0.7', '0.0.8', '0.0.9', '10.0'}

In [53]:
set(total_mdwarfs_ticids['S10'])

{'0.0', '0.0.5', '0.0.7', '0.0.8', '0.0.9', '10.0'}

In [54]:
mdwarfs_ticids = pd.read_csv('../../Research_course/Tic_files/total_mdwarfs_from_tic.csv', usecols = ['TICID', 'S10'],
                                     header = 0).astype(float).astype(str)
mdwarfs_ticids['TICID'] = mdwarfs_ticids['TICID'].astype(float)

mdwarfs_ticids = mdwarfs_ticids[mdwarfs_ticids['TICID'].isin(total_mdwarfs_ticids['TICID'])].reset_index(drop=True)
mdwarfs_ticids = mdwarfs_ticids.sort_values(['TICID']).reset_index(drop=True)
mdwarfs_ticids                                                                                                                                  
                                                                                                                                          
                                                                                                                                          

,TICID,S10
0,5.600000e+01,0.0
1,2.330000e+02,0.0
2,2.840000e+02,0.0
3,3.450000e+02,0.0
4,4.010000e+02,0.0
...,...,...
3557153,2.056092e+09,0.0
3557154,2.056093e+09,0.0
3557155,2.056103e+09,0.0
3557156,2.056106e+09,0.0


In [55]:
final = total_mdwarfs_ticids.merge(mdwarfs_ticids, on='TICID')

In [56]:
final[final['S10_x'].str.split('.').str[0].astype(float)>0.]

,TICID,S10_x,S10_y
14281,1.627658e+06,10.0,10.0
14282,1.627675e+06,10.0,10.0
14283,1.627688e+06,10.0,10.0
14284,1.627786e+06,10.0,10.0
14285,1.627841e+06,10.0,10.0
...,...,...,...
3492037,1.056143e+09,10.0,10.0
3492076,1.057276e+09,10.0,10.0
3492077,1.057288e+09,10.0,10.0
3492078,1.057307e+09,10.0,10.0


In [57]:

mdwarfs_ticids = mdwarfs_ticids[mdwarfs_ticids['TICID'].isin(total_mdwarfs_ticids['TICID'])].reset_index(drop=True)
mdwarfs_ticids = mdwarfs_ticids.sort_values(['TICID']).reset_index(drop=True)
mdwarfs_ticids                                                                                                                                  
                                                                                                                                          
                                                                                                                                          

,TICID,S10
0,5.600000e+01,0.0
1,2.330000e+02,0.0
2,2.840000e+02,0.0
3,3.450000e+02,0.0
4,4.010000e+02,0.0
...,...,...
3557153,2.056092e+09,0.0
3557154,2.056093e+09,0.0
3557155,2.056103e+09,0.0
3557156,2.056106e+09,0.0


In [58]:
float(list(set(total_mdwarfs_ticids['S10'].astype('str')))[0].split('.')[0])

10.0

In [ ]:
# rows = total_mdwarfs_ticids[~(total_mdwarfs_ticids['TICID']=='TICID').astype(bool)].astype(float).drop_duplicates().index.values.tolist()
# len(new_stuff) == len(total_mdwarfs_df)

In [ ]:
# rows = new_stuff.drop_duplicates().index.values.tolist()

In [ ]:
chunksize = 100000
inter = np.ceil(len(total_mdwarfs_ticids)/chunksize)
dataframe = pd.read_csv('./Total_Mdwarf_files/final_mdwarf_params.csv', 
                        chunksize = chunksize,
                        index_col = None,
                        skiprows = notrows, header = 0)

bar_format = "{desc}{percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} targets | {elapsed}<{remaining}"
pbar = tqdm(total=inter, smoothing=0.3,  position=0, leave=True, bar_format=bar_format)

jjj=0
lst_df = []
for chunk in dataframe:
#     df = chunk[chunk.index.isin(rows)].astype('str').reset_index(drop=True)
#     lst_df.append(chunk)
    chunk.to_csv('./Total_Mdwarf_files/only_mdwarf_params/mdwarfs_num_'+str(jjj), index=False)
    
    jjj+=1
    pbar.update(1)
    pass
# lst_df.to_csv("./Total_Mdwarf_files/limited_only_mdwarf_params.csv", index = False)

In [ ]:
# columns = ['TICID', 'RA', 'DEC', 'Vmag', 'Jmag', 'Tmag', 'Teff', 'eTeff_x', 'Rad',
#        'eRad', 'Mass', 'eMass', 'Rho', 'eRho', 'Dist', 'eDist', 'ContRatio',
#        'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11',
#        'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'S21',
#        'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31',
#        'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41',
#        'S42', 'S43', 'S44', 'S45', 'S46', 'S47', 'S48', 'S49', 'S50', 'S51',
#        'S52', 'S53', 'S54', 'S55', 'eTeff_y', 'logg', 'elogg', 'M/H', 'eM/H',
#        'GAIA', 'aLSM', 'bLSM']

In [ ]:
for i in range(1,14):
    print(i)
    filename = './Total_Mdwarf_files/by_sector/PM_year_1/mdwarf_params_sector_'+str(i)
    df = pd.read_csv(filename, 
                header = 0, index_col = 0)
    df['S'+str(i)]=10.0
    df.to_csv(filename, index = False)
    print('Done')


for i in range(14,27):
    print(i)
    filename = './Total_Mdwarf_files/by_sector/PM_year_2/mdwarf_params_sector_'+str(i)
    df = pd.read_csv(filename, 
                header = 0, index_col = 0)
    df['S'+str(i)]=10.0
    df.to_csv(filename, index = False)
    print('Done')



In [ ]:
# len(set(new_stuff['TICID'].astype(float)))#==set(total_mdwarfs_df['TICID'].astype(float))

In [ ]:
output_file = "./Total_Mdwarf_files/all_mdwarf_params.csv"
# CHUNK_SIZE = 5000
CHUNK_SIZE = 100000



total_mdwarfs = pd.read_csv('./Total_Mdwarf_files/final_mdwarf_params_new.csv', 
                                   chunksize = CHUNK_SIZE,
                                     header = 0, index_col = None)
bool_jjj = True
#     print(i)
for chunk in total_mdwarfs:
    for i in range(1, 55):
        chunk['S'+str(i)] = chunk['S'+str(i)].astype(str).str.split('.').str[0].astype(float)
        new_chunk = match_logg_and_teff_for_LDC(chunk)
        print('please let this work', new_chunk)
        if bool_jjj:
#             print('i am terribly unhappy', new_chunk)

            new_chunk.to_csv(output_file, mode="a", index=False)
            bool_jjj=False
        else:
            print('ugh')
            new_chunk.to_csv(output_file, mode="a", index=False, header=False)
#     mdwarfs = pd.concat(
#             [chunk[chunk['S'+str(i)].astype(str).str.split('.').str[0].astype(float)>0.] 
#     for chunk in total_mdwarfs_ticids]).sort_values(by='TICID').astype('str').reset_index(drop=True)
#     mdwarfs.to_csv('./Total_Mdwarf_files/by_sector/PM_year_2/mdwarf_params_sector_'+str(i))
#     print('done')

  0%|          | 0/100000 targets | 00:00<?

         aLSM    bLSM
0      0.1555  0.4459
1      0.3390  0.2994
2      0.1991  0.3902
3      0.1604  0.4325
4      0.1604  0.4325
...       ...     ...
99995  0.1991  0.3902
99996  0.1991  0.3902
99997  0.1671  0.4201
99998  0.1991  0.3902
99999  0.2315  0.3808

[100000 rows x 2 columns]
please let this work              TICID          RA        DEC       Vmag       Jmag       Tmag  \
0      322307010.0    0.185929 -88.174247  19.558500  15.561000  17.098101   
1      322307023.0    0.152830 -88.286010  19.282499  16.240000  17.752001   
2      322307055.0    0.342055 -88.540550  19.523001  16.240000  17.843800   
3      322307100.0    0.063819 -89.064984  19.708200  16.049999  17.487400   
4      325527324.0    0.745749 -88.457641  21.361900  16.796000  18.929600   
...            ...         ...        ...        ...        ...        ...   
99995  842564690.0  144.007376 -84.086567  20.433901        NaN  18.669100   
99996  842564811.0  144.470563 -84.048932  19.863600        NaN 

  0%|          | 0/100000 targets | 00:00<?

         aLSM    bLSM
0      0.1555  0.4459
1      0.3390  0.2994
2      0.1991  0.3902
3      0.1604  0.4325
4      0.1604  0.4325
...       ...     ...
99995  0.1991  0.3902
99996  0.1991  0.3902
99997  0.1671  0.4201
99998  0.1991  0.3902
99999  0.2315  0.3808

[100000 rows x 2 columns]
please let this work              TICID          RA        DEC       Vmag       Jmag       Tmag  \
0      322307010.0    0.185929 -88.174247  19.558500  15.561000  17.098101   
1      322307023.0    0.152830 -88.286010  19.282499  16.240000  17.752001   
2      322307055.0    0.342055 -88.540550  19.523001  16.240000  17.843800   
3      322307100.0    0.063819 -89.064984  19.708200  16.049999  17.487400   
4      325527324.0    0.745749 -88.457641  21.361900  16.796000  18.929600   
...            ...         ...        ...        ...        ...        ...   
99995  842564690.0  144.007376 -84.086567  20.433901        NaN  18.669100   
99996  842564811.0  144.470563 -84.048932  19.863600        NaN 

  0%|          | 0/100000 targets | 00:00<?

...

  0%|          | 0/100000 targets | 00:00<?

         aLSM    bLSM
0      0.1555  0.4459
1      0.3390  0.2994
2      0.1991  0.3902
3      0.1604  0.4325
4      0.1604  0.4325
...       ...     ...
99995  0.1991  0.3902
99996  0.1991  0.3902
99997  0.1671  0.4201
99998  0.1991  0.3902
99999  0.2315  0.3808

[100000 rows x 2 columns]
please let this work              TICID          RA        DEC       Vmag       Jmag       Tmag  \
0      322307010.0    0.185929 -88.174247  19.558500  15.561000  17.098101   
1      322307023.0    0.152830 -88.286010  19.282499  16.240000  17.752001   
2      322307055.0    0.342055 -88.540550  19.523001  16.240000  17.843800   
3      322307100.0    0.063819 -89.064984  19.708200  16.049999  17.487400   
4      325527324.0    0.745749 -88.457641  21.361900  16.796000  18.929600   
...            ...         ...        ...        ...        ...        ...   
99995  842564690.0  144.007376 -84.086567  20.433901        NaN  18.669100   
99996  842564811.0  144.470563 -84.048932  19.863600        NaN 

  0%|          | 0/100000 targets | 00:00<?

         aLSM    bLSM
0      0.1555  0.4459
1      0.3390  0.2994
2      0.1991  0.3902
3      0.1604  0.4325
4      0.1604  0.4325
...       ...     ...
99995  0.1991  0.3902
99996  0.1991  0.3902
99997  0.1671  0.4201
99998  0.1991  0.3902
99999  0.2315  0.3808

[100000 rows x 2 columns]
please let this work              TICID          RA        DEC       Vmag       Jmag       Tmag  \
0      322307010.0    0.185929 -88.174247  19.558500  15.561000  17.098101   
1      322307023.0    0.152830 -88.286010  19.282499  16.240000  17.752001   
2      322307055.0    0.342055 -88.540550  19.523001  16.240000  17.843800   
3      322307100.0    0.063819 -89.064984  19.708200  16.049999  17.487400   
4      325527324.0    0.745749 -88.457641  21.361900  16.796000  18.929600   
...            ...         ...        ...        ...        ...        ...   
99995  842564690.0  144.007376 -84.086567  20.433901        NaN  18.669100   
99996  842564811.0  144.470563 -84.048932  19.863600        NaN 

  0%|          | 0/100000 targets | 00:00<?

In [47]:
total_mdwarfs_ticids_new = pd.read_csv('./Total_Mdwarf_files/final_mdwarf_params_new.csv', usecols = ['TICID','S10'],
                                     header = 0).astype('str')
total_mdwarfs_ticids_new['TICID'] = total_mdwarfs_ticids_new['TICID'].astype(float)
total_mdwarfs_ticids_new = total_mdwarfs_ticids_new.sort_values(['TICID']).reset_index(drop=True)
len(total_mdwarfs_ticids_new)

3557159

In [48]:
total_mdwarfs_ticids = pd.read_csv('./Total_Mdwarf_files/final_mdwarf_params.csv', usecols = ['TICID','S10'],
                                     header = 0).astype('str')
total_mdwarfs_ticids['TICID'] = total_mdwarfs_ticids['TICID'].astype(float)
total_mdwarfs_ticids = total_mdwarfs_ticids.sort_values(['TICID']).reset_index(drop=True)
len(total_mdwarfs_ticids)

3557159

In [32]:
total_mdwarfs_ticids

,TICID,S10
0,322307010.0,0.0
1,322307023.0,0.0
2,322307055.0,0.0
3,322307100.0,0.0
4,325527324.0,0.0
...,...,...
3557509,252371374.0,0.0
3557510,284152076.0,0.0
3557511,55749832.0,0.0
3557512,376997810.0,0.0


In [7]:
total_mdwarfs_ticids[:127567+1]

,TICID,S10
0,56.0,0.0
1,233.0,0.0
2,284.0,0.0
3,345.0,0.0
4,401.0,0.0
...,...,...
127563,16138934.0,0.0
127564,16139035.0,0.0
127565,16139289.0,0.0
127566,16139307.0,0.0


In [4]:
for i in range(14,27):
    print(i)
    total_mdwarfs_ticids = pd.read_csv('./Total_Mdwarf_files/final_mdwarf_params.csv', 
                                       chunksize = 10000,
                                         header = 0, index_col = None)

    
    mdwarfs = pd.concat(
            [chunk[chunk['S'+str(i)].astype(float)>0.] 
    for chunk in total_mdwarfs_ticids]).sort_values(by='TICID').astype('str').reset_index(drop=True)
    mdwarfs.to_csv('./Total_Mdwarf_files/by_sector/PM_year_2/mdwarf_params_sector_'+str(i))

14


ValueError: could not convert string to float: '0.0.11'

In [ ]:
len(total_mdwarfs_ticids['TICID'].isin(list(total_mdwarfs_df['TICID'])))==len(total_mdwarfs_ticids['TICID'].isin(total_mdwarfs_df['TICID']))



In [ ]:
len(set(total_mdwarfs_df['TICID']))

In [ ]:
# some_mdwarfs = pd.read_csv('./Total_Mdwarf_files/final_mdwarf_params.csv', 
#                                nrows = 100000,
#                                  header = 0, index_col = None)
# set(list(some_mdwarfs['S1']))

In [ ]:
total_mdwarfs_ticids = pd.read_csv('./Total_Mdwarf_files/final_mdwarf_params.csv', 
                                   chunksize = 10000,
                                     header = 0, index_col = None)


mdwarfs = pd.concat(
        [chunk[chunk['S'+str(i)].astype(float)>0.] 
for chunk in total_mdwarfs_ticids]).sort_values(by='TICID').astype('str').reset_index(drop=True)
mdwarfs.to_csv('./Total_Mdwarf_files/by_sector/mdwarf_params_sector_'+str(i))

In [ ]:
total_mdwarfs_ticids.columns